<a href="https://colab.research.google.com/github/nataliechernysh/NEW_final/blob/main/final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Files


In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as pyplot
%matplotlib inline
!pip install ipympl
%matplotlib widget
from google.colab import output
output.enable_custom_widget_manager()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.2 MB/s eta 0:00:00


In [ ]:
from google.colab import files;
df=files.upload();
filename = list(df.keys ())[-1]
ds= pd.ExcelFile(filename)
sheetnames=ds.sheet_names
datasheet=pd.read_excel(filename, sheet_name=0); 
!rm{filename}
datasheet
total_data_sheet = np.array(datasheet)

Saving TEST.xls to TEST (1).xls
/bin/bash: rmTEST.xls: command not found


# Parse into dictionaries

In [ ]:
total_data_sheet = np.array(datasheet)

In [ ]:
def parse_headers(total_data_sheet):

  emg_headers = []
  accelerometer_headers = []
  vicon_headers = []

  emg_headers = total_data_sheet[3,:]
  print(emg_headers)

In [ ]:
print(parse_headers(total_data_sheet))

NameError: ignored

In [ ]:
def compile_accelerometer_data(total_data_sheet):
    #accelerometer data contains two data points per frame
    accelerometer_data = total_data_sheet[50626:54000,2:50]
    #print(accelerometer_data)
    temp_array = np.zeros(shape=(np.shape(accelerometer_data)[0]//2, 48))

    #goes through all of the accelerometer data and finds the average of each of
    #the two data points per frame
    count = 0
    dimensions = np.shape(accelerometer_data)

    for row in range(0,dimensions[0]-1,2):
        avg_array = np.sum(accelerometer_data[row:row+2,:],axis=0) / 2      
        temp_array[count] = avg_array
        count+=1
    accelerometer_data = temp_array

    #print(accelerometer_data[1][1])
    return accelerometer_data

In [ ]:
def compile_vicon_data(total_data_sheet):
  vicon_data = datasheet[54006:,2:119]
  return vicon_data

In [ ]:
def compile_emg_data(total_data_sheet):
    #emg data contains ten data points per frame
    delsys_emg_data = total_data_sheet[4:33744,2:18]
    
    #print(delsys_emg_data)
    
    temp_array = np.zeros(shape=(np.shape(delsys_emg_data)[0]//20, 16))   
    #print(np.sum(delsys_emg_data[0:20,:],axis=0) / 20)
    count = 0
    dimensions = np.shape(delsys_emg_data)
    #goes through all of the accelerometer data and finds the average of each of
    #the ten data points per frame
    for row in range(0,dimensions[0]-1,20):
        avg_array = np.sum(delsys_emg_data[row:row+20,:],axis=0) / 20      
        temp_array[count] = avg_array
        count+=1
   
    delsys_emg_data = temp_array
    
    return delsys_emg_data

In [ ]:
def parse_emg_to_dictionary(my_data, emg_list):
    emg_dict = {}
    for i in range(0, 16):
        emg_dict[emg_list[i]] = my_data[:, i]
    print(emg_dict.keys())

    return emg_dict

# GUI with tk


In [ ]:
import tkinter as tk
# Create the main window
root = tk.Tk()

# Add a label to the window
label = tk.Label(root, text="Hello, world!")
label.pack()

# Run the main loop
root.mainloop()

# Joint Angles (crediting [source](https://www.mdpi.com/1424-8220/22/19/7178) for equation )

In [ ]:
import math

# Define a function to calculate the angle between two vectors
def calculate_angle(vector1, vector2):
    dot_product = sum([vector1[i]*vector2[i] for i in range(len(vector1))])
    vector1_length = math.sqrt(sum([v**2 for v in vector1]))
    vector2_length = math.sqrt(sum([v**2 for v in vector2]))
    cos_theta = dot_product / (vector1_length * vector2_length)
    return math.acos(cos_theta)

# Define the vectors constituting the angles
vector_AB = [1, 0, 0]   # Vector from A to B
vector_BC = [0, 1, 0]   # Vector from B to C
vector_CD = [0, 0, 1]   # Vector from C to D
vector_DE = [0, 1, 1]   # Vector from D to E
vector_EF = [1, 0, 1]   # Vector from E to F
vector_FG = [1, 1, 0]   # Vector from F to G

# Calculate the angles (hip, knee, ankle joint)
angle_ABC = calculate_angle(vector_AB, vector_BC)
angle_CDE = calculate_angle(vector_CD, vector_DE)
angle_EFG = calculate_angle(vector_EF, vector_FG)

# Print the results
print(f"Angle ABC: {math.degrees(angle_ABC)} degrees")
print(f"Angle CDE: {math.degrees(angle_CDE)} degrees")
print(f"Angle EFG: {math.degrees(angle_EFG)} degrees")


# Box and Whisker Data Visualization for Hip and Knee angles (4 subplots swing, stance for each marker) [source](https://www.mdpi.com/1424-8220/22/19/7178) 